In [1]:
from moist_konrad import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

In [2]:
results_file = open("results_file_strong-low-atm-kluft-8.txt",'w')
lambda_file = open("lambda_file_strong-low-atm-kluft-8.txt",'w')
temp_file = open("temp_file_strong-low-atm-kluft-8.txt",'w')
water_file = open("water_file_strong-low-atm-kluft-8.txt",'w')

In [3]:
cases = [1.]
initial_temperatures = [270.,315.]
depths = [2.5,7.5]
solar_multiplier = [0.8,0.81,0.82,0.83,0.84,0.85,0.875,0.9,0.95,0.975,1.
                    ,1.01,1.02,1.03,1.04,1.05,1.06,1.07,1.08,1.09,1.1]
solar_multiplier = [0.8,1.,1.1]

initial_rh = [0.6,0.9]


land_atm_coupling = True
strong_coupling = True

albedo_sfc = 0.2

if strong_coupling == True:
    z_ref_T = 200
    z_ref_vmr = 2
else:
    z_ref_T = 2
    z_ref_vmr = 2

In [ ]:
%%time

for k in range(len(solar_multiplier)):
    for rhi in range(len(initial_rh)):
        for de in range(len(depths)):
            for te in range(len(initial_temperatures)):
                print(depths[de],initial_temperatures[te],solar_multiplier[k],initial_rh[rhi])
                ###PARAMETERS OF THE TEST:

                #flux for efficiency computation:
                # 1: LH + SH
                # 2: Radiation surface
                # 3: Atmosphere radiation
                case = cases[0]
                depth_d = depths[de]
                ini_rh = initial_rh[rhi]


                #coupled surface-atmosphere: 
                #if true, surface fluxes and surface temperature change at each step, constant otherwise
                coupled = land_atm_coupling

                SH_f = 15.
                LH_f = 140.
                Ts_f = np.array([initial_temperatures[te]])
                print(Ts_f)

                RLS = 1/2

                #Strong coupling surface-atmosphere:
                #If true the atmosphere moist adiabat starts at surface temperature
                #If false find the atmopsheric moist adiabat that conserves energy
                strong = strong_coupling

                popke = False
                if popke == True:
                    #solar v3
                    solar = 340.*0.9
                    angle = 0.

                    #solar v2,v5
                    solar = 510.*0.9
                    angle = 47.9

                    #solar v6
                    solar = 510.*0.95
                    angle = 47.9

                    #solar v7
                    solar = 510.* solar_multiplier[k]
                    angle = 47.9
                else:
                    solar = 480.* solar_multiplier[k]
                    angle = 42.05

                A = albedo_sfc

                ### INITIALIZATION
                E_imbalance = 0.

                plev, phlev = konrad.utils.get_pressure_grids(1000e2, 100, 128)
                atmosphere = konrad.atmosphere.Atmosphere(phlev)
                clearsky = konrad.cloud.ClearSky.from_atmosphere(atmosphere)
                surface = konrad.surface.SlabOcean(
                    temperature=288,
                    heat_sink=0,
                    depth = depth_d,
                    albedo = 0.5)

                radiation = konrad.radiation.RRTMG(solar_constant=solar, zenith_angle = angle)

                ndays = 10000
                timestep = 0.5 #days
                nsteps = int(ndays/timestep)

                printcounter = 0
                status = 0

                tol = 1e-5

                ### RCPE COMPUTATION

                surface['temperature'] = Ts_f
                surface.albedo = A
                atmosphere['T'][0] = moist_adiabat(surface['temperature'],atmosphere['T'][0],atmosphere)
                print(surface['temperature'],surface.albedo)

                conv_top = coldpoint(atmosphere['T'][0])
                RH = manabe_rh(ini_rh,atmosphere['plev'])
                atmosphere['H2O'][0] = rh_to_vmr(RH,atmosphere['T'][0],atmosphere['plev'],conv_top)
                T_atm_low = surface['temperature']

                print(surface['temperature'],surface.albedo)

                #### RCE FOR PRESENT-DAY CO2
                atmosphere['CO2'][0] = 348/1e6


                T_sfc_1 = [1e8]
                toa_1 = [1e8]
                for i in range(nsteps):
                    #surface fluxes
                    z = height(atmosphere['plev'],atmosphere['T'][0],
                               atmosphere['phlev'][0],surface['temperature'])
                    #zT = z_ref_T
                    #zvmr = z_ref_vmr

                    #T_ini,vmr_ini = T_vmr_z(T_atm_low,atmosphere['T'][0][0].copy(),
                     #                       atmosphere['H2O'][0,0].copy(),atmosphere['H2O'][0,1].copy(),
                      #                      z,zT,zvmr)


                    T_ini,vmr_ini = atmosphere['T'][0][0].copy(),atmosphere['H2O'][0,0].copy()

                    #compute the fluxes or input the forcing
                    if coupled == True:
                        LH = latent_heat(vmr_ini, T_ini,atmosphere['phlev'][0],RLS)
                        SH = sensible_heat(T_ini,surface['temperature'],atmosphere['phlev'][0],RLS)

                    else:
                        LH = LH_f
                        SH = SH_f


                    #convective adjustment of the armosphere
                    atmosphere,surface,radiation,net_rad_surface,atm_rad,\
                    T_atm_low,E_imbalance,prec_mass,prec_heating,\
                    RH,cold_point,prec_eff,water_content = RCPE_step_DSE(timestep,
                                                          atmosphere,surface,radiation,clearsky,
                                                          SH,LH,A,T_atm_low,
                                                          strong_coupling = strong,
                                                          constrain_RH = True,
                                                        Flux_case = case)



                    #update surface temperature with energetic fluxes (latent and radiative)
                    if coupled == True:
                        surface['temperature'] = surface['temperature'] + ((net_rad_surface - SH - LH)
                                               * seconds_day * timestep / surface.heat_capacity)

                    else:
                        surface['temperature'] = Ts_f



                    status += 1
                    if printcounter == 200:
                        print(status/2)
                    printcounter += 1

                    T_sfc_1.append(surface['temperature'])
                    toa_1.append(radiation['toa'].copy())
                    if abs(T_sfc_1[i]-T_sfc_1[i-1])<tol and abs(toa_1[i]-toa_1[i-1])<tol:
                        print(i)
                        break

                print('hehe')
                olr_rec = radiation['lw_flxu'][0,-1].copy() - radiation['lw_flxd'][0,-1].copy()
                net_sw = radiation['sw_flxd'][0,-1].copy() - radiation['sw_flxu'][0,-1].copy()
                isr_rec = radiation['sw_flxd'][0,-1].copy()
                results_file.write(str(coupled)+" "+str(depth_d)+" "+str(case)+" "+str(Ts_f)
                                   +" "+str(surface['temperature'])+" "+str(T_ini)
                                   +" "+str(net_rad_surface)+" "+str(atm_rad)+" "+str(radiation['toa'])
                                   +" "+str(olr_rec)+" "+str(net_sw)+" "+str(isr_rec)
                                   +" "+str(E_imbalance)
                                   +" "+str(LH)+" "+str(SH)+" "+str(prec_eff)
                                   +" "+str(prec_mass)+" "+str(LH/Lv * seconds_day)
                                   +" "+str(RH[0])+" "+str(water_content)+" "+str(ini_rh)
                                   +" "+str(atmosphere['CO2'][0][0])
                                   +"\n")

                if de == 1 and te == 0:
                    np.savetxt(temp_file,atmosphere['T'][0].reshape(1, -1).copy())
                    np.savetxt(water_file,atmosphere['H2O'][0].reshape(1, -1).copy())

                P1 = prec_mass
                q1 = atmosphere['H2O'][0].copy()[0]
                RH1 = RH[0]
                
                print(RH1, radiation['toa'], E_imbalance)

                #### RCE FOR PRESENT-DAY CO2
                atmosphere['CO2'][0] *= 2


                T_sfc_2 = [1e8]
                toa_2 = [1e8]
                olr_2 = [1e8]
                for i in range(nsteps):
                    #surface fluxes
                    z = height(atmosphere['plev'],atmosphere['T'][0],
                               atmosphere['phlev'][0],surface['temperature'])
                    zT = z_ref_T
                    zvmr = z_ref_vmr

                    #T_ini,vmr_ini = T_vmr_z(T_atm_low,atmosphere['T'][0][0].copy(),
                     #                       atmosphere['H2O'][0,0].copy(),atmosphere['H2O'][0,1].copy(),
                      #                      z,zT,zvmr)


                    T_ini,vmr_ini = atmosphere['T'][0][0].copy(),atmosphere['H2O'][0,0].copy()

                    #compute the fluxes or input the forcing
                    if coupled == True:
                        LH = latent_heat(vmr_ini, T_ini,atmosphere['phlev'][0],RLS)
                        SH = sensible_heat(T_ini,surface['temperature'],atmosphere['phlev'][0],RLS)

                    else:
                        LH = LH_f
                        SH = SH_f


                    #convective adjustment of the armosphere
                    atmosphere,surface,radiation,net_rad_surface,atm_rad,\
                    T_atm_low,E_imbalance,prec_mass,prec_heating,\
                    RH,cold_point,prec_eff,water_content = RCPE_step_DSE(timestep,
                                                          atmosphere,surface,radiation,clearsky,
                                                          SH,LH,A,T_atm_low,
                                                          strong_coupling = strong,
                                                          constrain_RH = True,
                                                        Flux_case = case)



                    #update surface temperature with energetic fluxes (latent and radiative)
                    if coupled == True:
                        surface['temperature'] = surface['temperature'] + ((net_rad_surface - SH - LH)
                                               * seconds_day * timestep / surface.heat_capacity)

                    else:
                        surface['temperature'] = Ts_f



                    status += 1
                    if printcounter == 200:
                        print(status/2)
                    printcounter += 1

                    T_sfc_2.append(surface['temperature'])
                    toa_2.append(radiation['toa'].copy())
                    olr_2.append(radiation['lw_flxu'][0,-1].copy() - radiation['lw_flxd'][0,-1].copy())

                    if abs(T_sfc_2[i]-T_sfc_2[i-1])<tol and abs(toa_2[i]-toa_2[i-1])<tol:
                        print(i)
                        break

                print('hehe2')
                olr_rec = radiation['lw_flxu'][0,-1].copy() - radiation['lw_flxd'][0,-1].copy()
                net_sw = radiation['sw_flxd'][0,-1].copy() - radiation['sw_flxu'][0,-1].copy()
                isr_rec = radiation['sw_flxd'][0,-1].copy()
                results_file.write(str(coupled)+" "+str(depth_d)+" "+str(case)+" "+str(Ts_f)
                                   +" "+str(surface['temperature'])+" "+str(T_ini)
                                   +" "+str(net_rad_surface)+" "+str(atm_rad)+" "+str(radiation['toa'])
                                   +" "+str(olr_rec)+" "+str(net_sw)+" "+str(isr_rec)
                                   +" "+str(E_imbalance)
                                   +" "+str(LH)+" "+str(SH)+" "+str(prec_eff)
                                   +" "+str(prec_mass)+" "+str(LH/Lv * seconds_day)
                                   +" "+str(RH[0])+" "+str(water_content)+" "+str(ini_rh)
                                   +" "+str(atmosphere['CO2'][0][0])
                                   +"\n")

                if de == 1 and te == 0:
                    np.savetxt(temp_file,atmosphere['T'][0].reshape(1, -1).copy())
                    np.savetxt(water_file,atmosphere['H2O'][0].reshape(1, -1).copy())

                P2 = prec_mass
                q2 = atmosphere['H2O'][0].copy()[0]
                RH2 = RH[0]

                print(RH2, radiation['toa'], E_imbalance)
                
                T1 = T_sfc_1[-1]
                T2 = np.array(T_sfc_2[1:])
                toa2 = np.array(toa_2[1:])
                olr2 = np.array(olr_2[1:])


                T_anom = T2 - T1

                lambda_1 = linregress( T_anom[200:,0], toa2[200:,0]).slope
                lambda_2 = linregress( T_anom[200:,0], olr2[200:]).slope


                lambda_file.write(str(coupled)+" "+str(depth_d)+" "+str(case)+" "+str(Ts_f)
                                 +" "+str(olr_rec)+" "+str(net_sw)+" "+str(isr_rec)
                                 +" "+str(T_anom[-1])+" "+str(lambda_1)+" "+str(lambda_2)
                                 +" "+str((P2-P1)/P1*100/T_anom[-1])
                                 +" "+str((q2-q1)/q1*100/T_anom[-1])
                                 +" "+str(np.max(toa2))+" "+str(np.min(olr2))
                                 +"\n")
    


2.5 270.0 0.8
[270.]
[270.] 0.2
[270.] 0.2
100.5
1074
hehe
0.5312725328669553 [-0.00216677] [-0.01398475]
977
hehe2
0.5457695362973602 [0.00275307] [0.00485976]
2.5 315.0 0.8
[315.]
[315.] 0.2
[315.] 0.2
100.5
1413
hehe
0.5312794808593726 [-0.00119325] [-0.00718097]
977
hehe2
0.545769559108328 [0.00274939] [0.00486076]
7.5 270.0 0.8
[270.]
[270.] 0.2
[270.] 0.2
100.5
2719
hehe
0.5313138984944579 [-0.00756051] [-0.00285132]
2351
hehe2
0.5457402264518275 [0.00746019] [0.0028515]
7.5 315.0 0.8
[315.]
[315.] 0.2
[315.] 0.2
100.5
3488
hehe
0.5313139615547302 [-0.00757377] [-0.00285434]
2351
hehe2
0.5457402265048203 [0.00746018] [0.00285149]
2.5 270.0 0.8
[270.]
[270.] 0.2
[270.] 0.2
100.5
1073
hehe
0.5312725578299764 [-0.00218148] [-0.01404228]
977
hehe2
0.5457695362210148 [0.00275309] [0.00485974]
2.5 315.0 0.8
[315.]
[315.] 0.2
[315.] 0.2


In [ ]:
temp_file.close()
water_file.close()
results_file.close()
lambda_file.close()

In [ ]:
print(T_anom[200:,0])

In [ ]:
plt.scatter(T_anom[:,0], olr2[:], s = 2)
plt.show()
plt.scatter(T_anom[:,0], toa2[:,0], s = 5)
plt.show()

In [ ]:
lambda_1 = linregress( T_anom[500:,0], toa2[500:,0]).slope
lambda_2 = linregress( T_anom[500:,0], olr2[500:]).slope
print(lambda_1,lambda_2)

In [ ]:
print(radiation['sw_flxd'][0,-1])

In [ ]:
print(plev)
print(phlev)
print(z)